In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from board_and_rules import Connect4, minimax
from td_agent import TDAgent, train_td_agent

RESULTADOS_FILE = "resultados.csv"


In [ ]:
def guardar_resultado(modo, ganador):
    try:
        df = pd.read_csv(RESULTADOS_FILE)
    except FileNotFoundError:
        df = pd.DataFrame(columns=["modo", "ganador"])

    df = pd.concat(
        [df, pd.DataFrame({"modo": [modo], "ganador": [ganador]})], ignore_index=True
    )
    df.to_csv(RESULTADOS_FILE, index=False)


In [ ]:
def graficar_resultados():
    try:
        df = pd.read_csv(RESULTADOS_FILE)
    except FileNotFoundError:
        print("No hay datos para graficar aún.")
        return

    modos_ordenados = ["1", "2", "3"]
    modo_labels = {
        "1": "1 - Jugador vs TDL",
        "2": "2 - TDL vs Minimax",
        "3": "3 - TDL vs Minimax+Poda",
    }
    ganadores = ["Jugador", "Minimax", "Empate", "TDL"]

    df["modo"] = df["modo"].astype(str).str.strip()
    df["ganador"] = df["ganador"].astype(str).str.strip()

    conteo = df.groupby(["modo", "ganador"]).size().unstack(fill_value=0)

    for ganador in ganadores:
        if ganador not in conteo.columns:
            conteo[ganador] = 0

    for modo in modos_ordenados:
        if modo not in conteo.index:
            conteo.loc[modo] = [0] * len(ganadores)

    conteo = conteo.loc[modos_ordenados, ganadores]
    xtick_labels = [modo_labels[modo] for modo in modos_ordenados]

    if (conteo.sum(axis=1) == 0).all():
        print("Aún no se han registrado partidas para graficar.")
        return

    ax = conteo.plot(kind="bar", figsize=(9, 6), width=0.7, colormap="viridis")
    plt.title("Resultados de las partidas")
    plt.xlabel("Modo de juego")
    plt.ylabel("Cantidad de victorias")
    plt.xticks(ticks=range(len(xtick_labels)), labels=xtick_labels, rotation=0)
    plt.legend(title="Ganador", labels=ganadores)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.savefig("grafica_resultados.png")
    plt.show()


In [ ]:
td_agent = TDAgent()
game = Connect4()

print("Entrenando agente TDL...")
train_td_agent(agent=td_agent, game=game, episodes=5000)
print("Entrenamiento finalizado.")


In [ ]:
def jugar_interactivamente(td_agent):
    game = Connect4()
    game.reset()

    print("\nElige el modo de juego:")
    print("1 - Jugador vs IA (TD Learning)")
    print("2 - IA (TDL) vs IA (Minimax sin poda)")
    print("3 - IA (TDL) vs IA (Minimax con poda)")

    mode = input("\nIngresa el número de la opción: ")

    turn = 1
    while True:
        game.print_board()
        player = 1 if turn % 2 != 0 else 2

        if mode == "1" and player == 1:
            try:
                column = int(input(f"Jugador {player}, elige una columna (1-7): ")) - 1
            except ValueError:
                print("Entrada inválida.")
                continue

            if column not in range(7) or not game.is_valid_location(column):
                print("Movimiento inválido.")
                continue

            game.drop_piece(column, player)
        else:
            if (mode == "1" and player == 2) or mode in ["2", "3


In [ ]:
jugar_interactivamente(td_agent)


In [ ]:
graficar_resultados()
